In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

In [2]:
data1 = pd.read_excel(r"output.xlsx")
print(data1.head(30))

    Unnamed: 0                                            Comment  Sentiment
0            0    مقداد وقداش مره تجيبوه افهمونا ماسط الشعب يحبوش          0
1            1               karim gharbi masit sitcom 3ejbetnich          0
2            2                                            mabldou          0
3            3  الهالكا مختصه ااقناه الزيتونه محايده وانتهت مه...          0
4            4  اتحاد الخراب والدمار خرب بلادنا يريدون افريقيا...          0
5            5             سلب لفلوسهم خاطر التوانسه وفيق بسرقتهم          0
6            6               المءسسات الخاصه مصانع يشملهم الاضراب          0
7            7                        يدخل المقروض الاضراب يامعلم          0
8            8                والخاصه اخاطر يخلصو زوز فرنك السميك          1
9            9                                  اتحاد الخراب نجاح          1
10          10  اطمع يهز للهاويه بلد شافه الافلاس والاتحاد هوي...          1
11          11  الاتحاد يتفاوظ الحكومه مداخيله الخاصه مشاكل ال...          0

In [3]:
df = pd.DataFrame(columns=('Comment','Sentiment'))
for ind in data1.index:
    ls=str(data1['Comment'][ind]).split()
    for i in range(len(ls)):
        if ls[i].isdecimal()==True or len(ls[i])<3:
            ls[i]=''
    print ('ls=',ls) 
    s=''
    for j in range(len(ls)):
        if ls[j]!='':
            s=s+' '+ls[j]
    #print(s)
    df.loc[ind]=[s,0]
    

ls= ['مقداد', 'وقداش', 'مره', 'تجيبوه', 'افهمونا', 'ماسط', 'الشعب', 'يحبوش']
ls= ['karim', 'gharbi', 'masit', 'sitcom', '3ejbetnich']
ls= ['mabldou']
ls= ['الهالكا', 'مختصه', 'ااقناه', 'الزيتونه', 'محايده', 'وانتهت', 'مهمتها', 'قانونيا']
ls= ['اتحاد', 'الخراب', 'والدمار', 'خرب', 'بلادنا', 'يريدون', 'افريقيا', 'العامل', 'التونسي', 'متعمل', 'بعقد', 'شركات', 'وهميه', 'الشركات', 'الاصليه', 'ببركه', 'فلعنه']
ls= ['سلب', 'لفلوسهم', 'خاطر', 'التوانسه', 'وفيق', 'بسرقتهم']
ls= ['المءسسات', 'الخاصه', 'مصانع', 'يشملهم', 'الاضراب']
ls= ['يدخل', 'المقروض', 'الاضراب', 'يامعلم']
ls= ['والخاصه', 'اخاطر', 'يخلصو', 'زوز', 'فرنك', 'السميك']
ls= ['اتحاد', 'الخراب', 'نجاح']
ls= ['اطمع', 'يهز', 'للهاويه', 'بلد', 'شافه', 'الافلاس', 'والاتحاد', 'هوي', 'المسءول', 'خراب', 'تونس', 'كملو', 'واصلو', 'الفوضي', 'تندمو', 'توانسا']
ls= ['الاتحاد', 'يتفاوظ', 'الحكومه', 'مداخيله', 'الخاصه', 'مشاكل', 'الدوله', 'متعمل']
ls= ['تشخر', 'زادت', 'بوف', 'يهلك', 'التحاد', 'الخراب']
ls= ['مره', 'مالزايد', 'كجيو', 'القحوا', 'اقلك'

ls= ['blastou', 'bsara7a', 'damou', 'ryzen']
ls= ['wmaset', 'lasit', 'karim', 'gharbi']
ls= ['ماسط', 'كالكلاسه', 'العمر', 'يضحك']
ls= ['يستحق', 'تعليق', 'معجبين', 'القامط']
ls= ['مالزايد', 'ظارب', 'مظروب', 'الهيكا', 'بطبيعتها', 'تتسبل', 'تكمل', 'تسكر', 'كلام', 'الناس', 'السبب', 'احترام', 'الضيف']
ls= ['منافق', 'مقال', 'كافون', 'منافقين', 'قبي']
ls= ['yéésir', 'masta', 'téfha', 'felekher', 'majééét', 'chay']
ls= ['ifadeid']
ls= ['تقعدوا', 'طول', 'عماركم', 'اعلام', 'عار', 'اوصلوا', 'لمستواه', 'تحكو']
ls= ['wmaset', 'laset']
ls= ['fdha', 'mostawète', 'bjèh', 'rabbi', 'ostrou', 'aardhkom', 'raa', 'iou', 'ennès', 'mayhemhomch', 'yshoufa', 'aarkom']
ls= ['بهامه']
ls= ['مستوي', 'المسلسل', 'للانحطاط']
ls= ['blidaaaaaaaaaaaaaaaaaaaa']
ls= ['ouiuuuuf', 'tfadded']
ls= ['مستوء', 'للانحطاط']
ls= ['سلسله', 'سخيفه', 'الممثل', 'بهذلو', 'بايديهم', 'اسخف', 'هكه']
ls= ['اشاله', 'ربي', 'يهلك']
ls= ['ثعلوب']
ls= ['يقدم', 'البرنامج', 'حمار', 'يحمل', 'اصفار']
ls= ['ممصتو', 'مايتهضمش']
ls= ['جراثيم', 'بلد']
l

ls= ['الشيطان', 'اشب', 'الخييييييييييت']
ls= ['wmaset', 'laaaaseta']
ls= ['wmaset', 'mleyketha']
ls= ['ryzen', 'mlayktek', 'mo9ded']
ls= ['merg']
ls= ['يليق', 'والبرامج']
ls= ['مسلسل', 'خامج', 'لعام', 'فيه', 'بنت', 'سامي', 'الفهري', 'تغتصب']
ls= ['3ejbetnich']
ls= ['جاه', 'عندو', 'همه', 'انسحب', 'سماع', 'الصوت', 'الشجي', 'لحمزه', 'الفضلاوي']
ls= ['الفاشله']
ls= ['عجبتكمش']
ls= ['عفشه']
ls= ['كعلاف']
ls= ['بغل', 'ناقصو', 'ذيل', 'مقال']
ls= ['فالارهاب', 'متنكر']
ls= ['kelb']
ls= ['امين', 'اتزيد', 'البهاما', 'قداشك', 'ابهيم', 'بالقوي', 'برجوليه']
ls= ['قعر']
ls= ['ملحد']
ls= ['مله', 'قعر']
ls= ['لعنه', 'شهره', 'المبارك']
ls= ['احتقرك', 'طاهر', 'حسين']
ls= ['فنانه', 'تاريخ', 'مستوي', 'اخلاق']
ls= ['تتواصل', 'الايحاءات', 'الجنسيه', 'الكلام', 'الفاحش', 'ماخورالدعاره', 'قلناها', 'نزال', 'مقال', 'ماخور', 'قناه', 'تلفزيه']
ls= ['يهلك']
ls= ['سخانه', 'وربي', 'يتولاك']
ls= ['وكاذب', 'مفتري', 'قاومت', 'الاستبداد', 'اشباه', 'الرجال', 'مرتزقه', 'ومنافقين']
ls= ['تستحقها', 'تستحق', 'الفاشله']
ls= ['ك

ls= ['عميل', 'بشار']
ls= ['مستوي', 'جاهل']
ls= ['رجل', 'عفاط']
ls= ['سيف', 'الدين', 'مصباحي', 'انبزتني', 'ماعبرت', 'مستواك', 'للانحطاط', 'كيفك', 'بطبيعه', 'انجمش', 'فنان']
ls= ['سيف', 'الدين', 'مصباحي', 'رجل', 'عيلتكم']
ls= ['يظهر', 'فيك', 'ليتعلم', 'الامهات', 'العزباوات', 'يعطك', 'وهف', 'يهزك', 'الهمل', 'والخمج', 'تحبوا', 'تنشروه', 'العباد', 'زباله']
ls= ['مريضه']
ls= ['ya3tek', 'wahef']
ls= ['استغفر', 'العظيم', 'واتوب']
ls= ['مستعرفه', 'الفاجره']
ls= ['ربي', 'يشفيها', 'مريضه', 'نفسيتها', 'مهتزه', 'تفقه', 'والمصالحه', 'للنصح']
ls= ['يكب', 'سعيد', 'عكست', 'اصبت']
ls= ['سود', 'لوجه', 'سالم', 'لسود']
ls= ['بعزاه', 'كلب', 'عرقت', 'كلوشار']
ls= ['سالم', 'الابيض', 'عمرو', 'كامل', 'عايشه', 'يحكي', 'الرجال', 'روح']
ls= ['bouter3aaa', 'ter3tk', 'mankouba']
ls= ['خطر', 'المجتمع', 'حيوان', 'وداعشي', 'خطير']
ls= ['وتحقير', 'واسقط', 'وانذل', 'عبير', 'سوي', 'خنازيرها', 'ومومساتها', 'يصفقولها']
ls= ['لعنه', 'لثنين']
ls= ['الزوز', 'خربه', 'بلد', 'تربحكم']
ls= ['لوجه', 'ريا', 'وسكينه']
ls= ['منين', 'ا

ls= ['ياخي', 'فرض', 'عليك', 'واجبه', 'يذكر', 'الناس', 'فلك', 'الاختيار', 'الجنه', 'النار', 'تحبي', 'نلعبو']
ls= ['كلام', 'الدكتور', 'الحق', 'النخبه', 'الفرنكفونيه', 'العلمانيه', 'يذكر', 'اسم', 'يهيج', 'ويميج']
ls= ['ربي', 'يحفضك', 'مالزايد', 'ماله', 'الجرذ', 'الحسود', 'يمثل', 'التوانسه', 'المتربيه', 'ولاد', 'الاوصول']
ls= ['5ouya', 'aswemhom']
ls= ['تطيح', 'مالعين']
ls= ['نكرهك', 'اسالي']
ls= ['تافه']
ls= ['يعلم', 'الولدين', 'مفيده', 'وحكايات', 'تونسيه', 'اباحيه', 'وجراه', 'ووقاحه', 'اغنيه', 'نجلا', 'ومناقشه', 'الموضوع', 'قناه', 'سامي', 'الفهري', 'يصبح', 'بالقاء', 'محاضره', 'الشرف', 'ماخور']
ls= ['ماسط', 'تفدلكش', 'خير']
ls= ['chakib', 'yshoufa', '9asmek', 'rebi', 'rek', 'mase9', 'man9ek', 'maset', 'masset']
ls= ['تمثلي', 'نساء', 'توانسا', 'dégage']
ls= ['t3ayaf']
ls= ['عزوزه', 'خارفه']
ls= ['مثير', 'للاشمءزااااز']
ls= ['فضيحه', 'تتحول', 'الكلمه', 'فضوحه', 'هههههه']
ls= ['اثقل', 'نهار', 'الاثنين']
ls= ['a7sen']
ls= ['عجبتكمش', 'محلها']
ls= ['عندو', 'الحق', 'بندير', 'مان']
ls= ['5îr', '

...

ls= ['افلام', 'مترجمه', 'والبرامج', 'بريطانيه', 'امريكيه', 'هنديه']
ls= ['be5lef', 'formations', 'mat7rmoch', 'aflem', 'ybda', 'ytfarej', 'yzid', 'sout', 'ysma3', 'ya9ra', 'klem', 'yched', 'meno', 'max', 'soghri', 'hedha', 'nfa3ni', '3al5er', 'surtt', 'tweli', 'accent', 'rabi', 'yfathel']
ls= ['احسن', 'ييتفرج', 'افلام', 'يركز', 'الكلام', 'كفاش', 'يحكو', 'طريقه']
ls= ['5alih', 'ytaba3', 'barsha', 'aflem', 'anglais', 'yasma3', 'wghneyetha']
ls= ['نتفكر', 'بروفه', 'نصحتني', 'بالshort', 'level', 'stories', 'موجودين', 'عالانترنت', 'زيدو', 'افلام', 'animation', 'يعمل', 'animes', 'ينجم', 'يسمع', 'بقرايتك', 'نفس']
ls= ['اسهل', 'طريقه', 'يدخل', 'اليوتوب', 'يحل', 'فيديوات', 'بالانقليزيه', 'فيه', 'سوتيتراج']
ls= ['ifarejha', 'aflem', 'ysm3', 'ghna', 'anglais', 'lazem', 'yrakz', 'lklem', 'y9olouh', 'videos', 'bahin', 'y9ra', 'yt3lm', 'lbaz', 'lfayda', 'y3rf']
ls= ['yestenees', 'ifarejha', '7ajet', 'anglais', 'titrage', 'anglais', 'pratique', 'a7ki', 'sal7oulou']
ls= ['ahaam', '7ajja', 'y3alem', 'm

ls= ['brbi', 'tnjm', 'tji', 'prv']
ls= ['haseb', 'naaref', 'tnajjem', 'tel9a', 'ykhaddem', 'behya', 'good', 'luck']
ls= ['9ritha', 'ne5ou', 'chharya', 'nemchy', 'ykhaddem', 'akhaw', 'nlawej', 'behiya', 'tunis', '5ater', '3omrk', 'mate5ou', '7a9k']
ls= ['9retha', 'bhya', 'tayra', 'lazem', 'tkml', 'feha', 'mt7bsech']
ls= ['yshoufa', 'matloub', 'sou9', 'cho8l', 'baadika', 'e5tar', 'taba3']
ls= ['propositions', 'bachelier', 'mat7ebouch', 'ya9ra', 'juste', 'licence', 'distance', 'maynjmch', 'yetna9l', 'dossier', 'médical']
ls= ['تنحط', 'الحربوشه', 'مفتفته', 'الفطور', 'ماغير', 'يشلق', 'واقعد', 'لحلح', 'بيه', 'يتوهم', 'شربتوش']
ls= ['الدوا', 'ياغورته', 'حليب', 'كيمه', 'متعمل', 'صغار', 'والاحسن', 'استشير', 'طبيب', 'نفسي', 'ظاهر', 'بوك', 'متقبل', 'مرضوا', 'عادي', 'ونمروا', 'بيه', 'يرضي', 'بقدر', 'وابتلاء', 'ربي', 'ويشرب', 'دواه', 'وربي', 'يشفيه', 'ويعافيه']
ls= ['ناس', 'فيبالهمش', 'فمه', 'site', 'web', 'متنجم', 'تلوج', 'فيه', 'اله', 'اكسجين', 'الولايه']
ls= ['ساهله', 'الحكايه', 'اختار', 'ولايتك

ls= ['اخوتك', 'تتحمل', 'المسءوليه', 'يعني', 'يتلاهاو', 'تغيب', 'بناو', 'حياتهم', 'تبنيها']
ls= ['شوف', 'freelance', 'مجال', 'بقرايتك', 'الترجمه', 'مناصات', 'عربيه', 'وفما', 'عالميه', 'upwork']
ls= ['خواتك', 'عرسوا', 'يعني', 'بوهم', 'المدينه', 'تنجم', 'خليه', 'سينو', 'مالزايد', 'حاول', 'فهمو', 'وضعيتك', 'قولهولو', 'الكلام', 'ربي', 'يهديه']
ls= ['chouflou', 'yetelhe', 'e9smou', 'chahreyet', '3abbd', 'khwetek', '3araslou', 'a9n3ou', 'iji', '7a9ek', 'tebni', '7yetek', 'tkawen', 'mosta9blek']
ls= ['حكيم', 'كلامك', 'لوجيك', 'الحق']
ls= ['يعطيك', 'وصحه', 'تخمم', 'بوك', 'يفرح', 'يراك', 'فرحانه', 'وناجحه']
ls= ['تمشي', 'الخدمه', 'روحك', 'ارض', 'واسعه', 'وتعملوا', 'تاليفون', 'تتصل', 'بيه', 'تتطمن', 'وخواتك', 'المعرسين', 'يتفقدوه', 'عندو', 'كيفك', 'بالضبط', 'ووقت', 'تركز', 'الخدمه', 'تولي', 'تروح', 'الويكاند', 'ماتقدر']
ls= ['نصيحه', 'خويا', 'عرس', 'لبوك', 'خاطر', 'العمر', 'مالزايد', 'صحته', 'تنقص', 'واي', 'راجل', 'يستحق', 'لمرا', 'خاصه', 'الكبر', 'وميمتك', 'ماشاد', 'بلاصتها', 'خلقه']
ls= ['ننصحك

ls= ['7awel', 't3rslouu', 't9alet', 'akid', 'tl9a', 'mra', 'tbda', '3aycha', 'w7dha', 't9bl']
ls= ['بنيه', 'لواش', 'عايشه', 'اخوانك', 'الاولاد']
ls= ['المهم', 'تنجم', 'الخدمه', 'الحلول', 'اقترحوها']
ls= ['ادخل', 'تعلم', 'الخدمه', 'بالانترنت', 'فريلانسار', 'عاليوتيوب', 'فيه', 'الموضوع', 'نادي', 'القراءه', 'العمليه']
ls= ['نفعش', 'وماعرفتش', 'تخدم', 'بالانترنت', 'ابعث', 'السيفي', 'قبلوك', 'خدمه', 'ستاج', 'اخوتك', 'الامر', 'للواقع']
ls= ['بوهم', 'البركه', 'صغير', 'رايي']
ls= ['فمه', 'مرا', 'تناسبوا', 'وتكون', 'جزء', 'الحل', 'بمشكلتك', 'وعبا', 'جديد', 'شوفلو', 'الحالات']
ls= ['نصيحه', 'بوك', 'هزو', 'وريه', 'تصاور', 'بلايص', 'متع', 'محواس', 'تنجم', 'تهزو', 'اقنعو', 'سياستكم', 'قلو', 'الخبزه']
ls= ['تنساش', 'العمر', 'يجري', 'هذاكه', 'تقعدش', 'تستني', 'الفرصه', 'تجيك', 'العمر', 'تعاود', 'غدوه']
ls= ['ربي', 'سبحانو', 'مقال', 'عليك', 'بالحركه', 'بالبركه', 'تسعي', 'تعديش', 'بقيه', 'عمرك', 'نادم', 'تتحسر', 'ضيعتو', 'فايده', 'شاء', 'لله', 'يرزقك']
ls= ['نعمه', 'تحب', 'اطيشها', 'خاتر', 'الدنيا']
ls

ls= ['medecine', 'faut', 'aimé', 'dois', 'faire', 'grand', 'sacrifices', 'hedha', 'réve', 'suit', 'voie', 'fékra', 'famille', 'logique', 'choses', 'sayeb']
ls= ['morale', 'choses', 'dit', 'mat7ebouch', 'jeune', 'avenir', 'a3tii', 'forsa', 'rouhek', 'matsm3a', 'bourses', 'campus', 'france']
ls= ['a3mel', 'doura', 'site', 'rakezz', 'oumour', 'orientation', 'ahém', '5arja', 'bourse', 'harba', 'temps', 'courage']
ls= ['تعملش', 'تندم', 'عمرك', 'كامل', 'تسمحلك', 'الخدمه', 'تحب', 'اوروبا', 'العام', 'التوانسه', 'نجحوا', 'مناضره', 'المعادله', 'فرنسا', 'المانيا', 'تقبل', 'اللهم', 'تحبهاش', 'اعمل', 'تحضيري', 'بعدها', 'اختار', 'اختصاص', 'هندسي', 'تحبو', 'ربي', 'يدلك', 'للخير']
ls= ['voies', 'yejmou', 'yhezzouk', 'lel', 'barra']
ls= ['domaines', 'b3ad', 'lezmek', 'takhtar', 'haja', 'maghrouma']
ls= ['parcours', 'maghrouma', 'math', 'info', 'coté', 'takhdim', 'mokh', 'algorithme', 'jdid', 'tefrah', 'temchilek', 'exécution', 'shiha']
ls= ['kammel', 'domaine', 'maghrouma', 'surtt', 'problèmes', 'thebb

In [4]:
df

,Comment,Sentiment
0,مقداد وقداش مره تجيبوه افهمونا ماسط الشعب يحبوش,0
1,karim gharbi masit sitcom 3ejbetnich,0
2,mabldou,0
3,الهالكا مختصه ااقناه الزيتونه محايده وانتهت م...,0
4,اتحاد الخراب والدمار خرب بلادنا يريدون افريقي...,0
5,سلب لفلوسهم خاطر التوانسه وفيق بسرقتهم,0
6,المءسسات الخاصه مصانع يشملهم الاضراب,0
7,يدخل المقروض الاضراب يامعلم,0
8,والخاصه اخاطر يخلصو زوز فرنك السميك,0
9,اتحاد الخراب نجاح,0


In [5]:
data1['Comment']=df['Comment']

In [6]:
data1

,Unnamed: 0,Comment,Sentiment
0,0,مقداد وقداش مره تجيبوه افهمونا ماسط الشعب يحبوش,0
1,1,karim gharbi masit sitcom 3ejbetnich,0
2,2,mabldou,0
3,3,الهالكا مختصه ااقناه الزيتونه محايده وانتهت م...,0
4,4,اتحاد الخراب والدمار خرب بلادنا يريدون افريقي...,0
5,5,سلب لفلوسهم خاطر التوانسه وفيق بسرقتهم,0
6,6,المءسسات الخاصه مصانع يشملهم الاضراب,0
7,7,يدخل المقروض الاضراب يامعلم,0
8,8,والخاصه اخاطر يخلصو زوز فرنك السميك,1
9,9,اتحاد الخراب نجاح,1


## STEP 4: Création du modèle de Machine Learning

### Deep Learning model with CNN

In [7]:
import pandas as pd
import numpy as np
import re
import nltk
import tensorflow
from nltk.corpus import stopwords
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
#from keras.models import Sequential
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import Activation, Dropout, Dense
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import GlobalMaxPooling1D
from tensorflow.python.keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [8]:
feature = data1.Comment.fillna(' ')
target = data1.Sentiment
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size=0.2, random_state=42, shuffle=True)

In [9]:
from keras.layers import  Dropout, Dense
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from keras import layers
from sklearn import metrics

In [37]:
def TFIDF(X_train, X_test,MAX_NB_WORDS=10000):
    vectorizer = TfidfVectorizer(max_features=MAX_NB_WORDS)
    vectors = vectorizer.fit_transform(X_train)
    indices = np.argsort(vectorizer.idf_)[::-1]
    features = vectorizer.get_feature_names()
    top_n = 10
    top_features = [features[i] for i in indices[:top_n]]
    print(features)
    X_train = vectorizer.fit_transform(X_train).toarray()
    X_test = vectorizer.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)


In [38]:
def Build_Model_DNN_Text(shape, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 40 # number of nodes parfois 64
    nLayers = 2 # number of  hidden layer  2 per default

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(4, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


In [39]:
X_train_tfidf,X_test_tfidf = TFIDF(X_train,X_test)

['ﻻيتناسب', 'خيييييتت', 'خيييت', 'خيوطك', 'خيريه', 'خيتتتت', 'خيانته', 'خوياااااا', 'خوياا', 'خول']
tf-idf with 10000 features


In [34]:
X_train_tfidf[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [35]:
X_train_tfidf.shape

(8320, 10000)

In [36]:
model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1])
model_DNN.summary()
exit(1)
model_DNN.fit(X_train_tfidf, Y_train,
                              validation_data=(X_test_tfidf, Y_test),
                              validation_split=0.2,
                              epochs=100,
                              batch_size=128,
                              verbose=2,shuffle=True)

predicted = model_DNN.predict_classes(X_test_tfidf)

print(metrics.classification_report(Y_test, predicted))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 40)                400040    
_________________________________________________________________
dropout_10 (Dropout)         (None, 40)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 40)                1640      
_________________________________________________________________
dropout_11 (Dropout)         (None, 40)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 40)                1640      
_________________________________________________________________
dropout_12 (Dropout)         (None, 40)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 4)                

Epoch 74/100
 - 1s - loss: 0.1614 - accuracy: 0.9220 - val_loss: 2.3062 - val_accuracy: 0.7096
Epoch 75/100
 - 1s - loss: 0.1691 - accuracy: 0.9239 - val_loss: 2.2512 - val_accuracy: 0.7067
Epoch 76/100
 - 1s - loss: 0.1631 - accuracy: 0.9218 - val_loss: 2.2643 - val_accuracy: 0.7096
Epoch 77/100
 - 1s - loss: 0.1586 - accuracy: 0.9246 - val_loss: 2.3535 - val_accuracy: 0.7120
Epoch 78/100
 - 1s - loss: 0.1613 - accuracy: 0.9256 - val_loss: 2.4122 - val_accuracy: 0.7135
Epoch 79/100
 - 1s - loss: 0.1549 - accuracy: 0.9272 - val_loss: 2.4645 - val_accuracy: 0.7149
Epoch 80/100
 - 1s - loss: 0.1645 - accuracy: 0.9231 - val_loss: 2.3992 - val_accuracy: 0.7115
Epoch 81/100
 - 1s - loss: 0.1619 - accuracy: 0.9206 - val_loss: 2.4683 - val_accuracy: 0.7183
Epoch 82/100
 - 1s - loss: 0.1656 - accuracy: 0.9216 - val_loss: 2.3550 - val_accuracy: 0.7144
Epoch 83/100
 - 1s - loss: 0.1618 - accuracy: 0.9232 - val_loss: 2.4294 - val_accuracy: 0.7111
Epoch 84/100
 - 1s - loss: 0.1607 - accuracy: 0.92

In [ ]:
X_train_tfidf.shape

In [ ]:
Y_train.shape

In [ ]:
X_test_tfidf

In [ ]:
Y_test

In [44]:
seed = 7
np.random.seed(seed)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

### Using a CNN model

In [55]:
# Some handy libraries to build a baseline CNN model
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.layers import Activation, GlobalMaxPooling1D
from keras import optimizers
from keras.layers.embeddings import Embedding

In [56]:
from numpy import newaxis

def baseline_cnn_model(fea_matrix, n_class, mode, compiler):
  # create model
    model = Sequential()
    model.add(Conv1D(64,3, activation = 'relu'))
    model.add(MaxPool1D(pool_size = 2))
    model.add(Conv1D(filters=128, kernel_size = (3), activation = 'relu'))
    model.add(MaxPool1D(pool_size = 2))
    model.add(Flatten())
    model.add(Activation('relu'))
    model.add(Dense(n_class, activation = 'softmax'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop',  metrics=['acc'])
    #model_ptw2v.add(Dense(1, activation='sigmoid'))

    return model

In [57]:
# Tuning hyper-parameters
import math
print('shape=',X_train_tfidf.shape[1])
lr = math.exp(-3)
batch_size=128
num_epochs=100
decay= math.exp(-4)
mode="reg"
n_class=4 #5
adm=optimizers.Adam(lr=lr,decay=decay)
sgd=optimizers.SGD(lr=lr, nesterov=True,momentum=0.7,decay=decay)
Nadam= optimizers.Nadam(lr=lr, beta_1=0.9,beta_2=0.999,epsilon=math.exp(-8))

model=baseline_cnn_model(X_train_tfidf,n_class,mode,Nadam)
history = model.fit(X_train_tfidf, Y_train,epochs=10, 
                    validation_data=(X_test_tfidf, Y_test))


shape= 26392


ValueError: Input 0 is incompatible with layer conv1d_1: expected ndim=3, found ndim=2

In [ ]:

# Model Training
history=model.fit(X_train_tfidf,Y_train,batch_size=batch_size,epochs=num_epochs,verbose=1,validation_split=0.2)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
input_shape = X_train_tfidf.shape  
model.build(input_shape) 

print(model.summary())

In [ ]:
history = model.fit(X_train_tfidf, Y_train, batch_size=128, epochs=30, verbose=1, validation_split=0.2)

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=1)

In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

### Deep Learning models using an LSTM

In [58]:
# Import the tools needed from keras
import keras.backend as K
#from keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.layers import Dense, Embedding, LSTM
#from keras.models import Sequential
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [59]:
# Initialize and fit the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [60]:
# Use that tokenizer to transform the text messages in the training and test sets
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [61]:
# Pad the sequences so each sequence is the same length
X_train_seq_padded = pad_sequences(X_train_seq, 50)
X_test_seq_padded = pad_sequences(X_test_seq, 50)

In [62]:
model = Sequential()

In [63]:
model.add(Embedding(len(tokenizer.word_index)+1, 32))
model.add(LSTM(32, dropout=0, recurrent_dropout=0))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          845952    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 855,361
Trainable params: 855,361
Non-trainable params: 0
_________________________________________________________________


In [64]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [65]:
Y_test

7061     2
2709     0
1217     2
35       0
8920     3
9900     3
9325     3
4226     1
8775     3
4560     2
1458     2
1660     2
7823     3
8418     2
6603     0
3006     1
9534     3
6066     0
2742     0
311      0
5419     0
9246     3
10203    3
6054     0
439      0
2484     0
2769     0
39       0
5066     0
2680     0
        ..
3728     1
6461     1
8638     3
2919     0
5228     1
4640     2
3118     1
7212     2
7508     2
4881     1
5526     0
9938     3
2271     0
1109     2
6664     0
6492     1
1038     2
20       0
5580     0
2495     0
4562     2
7949     3
934      2
6373     0
2955     1
394      0
4913     1
1737     2
561      0
1162     2
Name: Sentiment, Length: 2079, dtype: int64

In [66]:
# Fit the RNN model
history = model.fit(X_train_seq_padded, Y_train, 
                    batch_size=32, epochs=20,
                    validation_data=(X_test_seq_padded, Y_test.values))

Train on 8315 samples, validate on 2079 samples
Epoch 1/20
8315/8315 [==============================] - 22s 3ms/sample - loss: -25.5264 - acc: 0.2463 - val_loss: -63.9894 - val_acc: 0.2650
Epoch 2/20
8315/8315 [==============================] - 20s 2ms/sample - loss: -136.3987 - acc: 0.2462 - val_loss: -213.2073 - val_acc: 0.2650
Epoch 3/20
8315/8315 [==============================] - 20s 2ms/sample - loss: -340.7050 - acc: 0.2462 - val_loss: -451.1419 - val_acc: 0.2650
Epoch 4/20
8315/8315 [==============================] - 19s 2ms/sample - loss: -631.0549 - acc: 0.2462 - val_loss: -765.3185 - val_acc: 0.2650
Epoch 5/20
8315/8315 [==============================] - 20s 2ms/sample - loss: -1002.8105 - acc: 0.2462 - val_loss: -1159.5187 - val_acc: 0.2650
Epoch 6/20
8315/8315 [==============================] - 20s 2ms/sample - loss: -1452.3197 - acc: 0.2462 - val_loss: -1619.6822 - val_acc: 0.2650
Epoch 7/20
1472/8315 [====>.........................] - ETA: 15s - loss: -1641.0090 - acc: 0

KeyboardInterrupt: 

# Deep Learning Model with CNN

In [168]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

X = data1.Comment.fillna(' ')
Y = data1.Sentiment

label_encoder = LabelEncoder()

Y = label_encoder.fit_transform(Y)

Y = Y.reshape(-1, 1)
Y

array([[0],
       [0],
       [0],
       ...,
       [3],
       [3],
       [3]], dtype=int64)

In [169]:
# Program to find most frequent 
# element in a list
  
from collections import Counter
  
def most_frequent(List):
    occurence_count = Counter(List)
    #return occurence_count.most_common(1)[0][0]
    return occurence_count.most_common(1)
  
vocab= set(" ".join(X).split())
print(most_frequent(vocab))

[('ماتعطيكش', 1)]


In [170]:
def word_count(voc):
    counts = dict()
    

    for word in voc:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

vocab= set(" ".join(X).split())
#print( word_count(vocab))

In [171]:
X

0           مقداد قداش مره تجيبوه افهمونا ماسط الشعب يحبوش
1                   karim gharbi masit i sitcom ma3jibnich
2                                                  mabldou
3        الهالكا مختصه قناه الزيتونه محايده وانتهت مهمت...
4        اتحاد الخراب والدمار خرب بلادنا يريدون افريقيا...
5                   سلب لفلوسهم خاطر التوانسه فاقو بسرقتهم
6                   المءسسات الخاصه المصانع يشملهم الاضراب
7                              مدخل المقروض الاضراب يامعلم
8                   والخاصه طز اخاطر يخلصو زوز فرنك السميك
9                                        اتحاد الخراب ينجح
10       اطمع يهز للهاويه بلاد شافه الافلاس والاتحاد هو...
11       الاتحاد يتفاوظ الحكومه مداخيله الخاصه حل مشاكل...
12                        تشخر زادت بوف يهلك التحاد الخراب
13                مره ازيدنا كجيو القحوا اقلك نفاذ المخزون
14       الكميه ناقصه مشطلع سومها الحمد لله مرضنا وبرين...
15       تفدلك 6 ملاين تتحصل عامين اطيشها الزبله الشعب ...
16                                              لقاح مشك

In [172]:


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)

vocab= set(" ".join(X_train).split())
max_words = len(set(" ".join(X_train).split()))
max_len = X_train.apply(lambda x: len(x)).max()

print(max_words)
print(max_len)
print(len(vocab))
#print(vocab)



28128
4430
28128
{'ماتعطيكش', 'شوكات', 'نطاق', 'تفسخني', 'فريال', 'ريف', 'as2aliii', 'عمي', 'ياحاوين', 'tawer', 'للرسول', 'المولي', 'mehadhamtouch', 'عصك', 'لمين', 'الابيض', 'rebi', 'الدخول', 'za3ma', 'شغل', 'éfe9', 'صانع', 'قيمت', 'leb3a4hom', 'الدينار', 'وحبيبنا', 'ولاقوه', 'الثغرات', 'imha', 'partage', 'سرعه', 'للرجوله', 'avantage', 'الvariant', 'مالخر', 'ينصحوها', 'اراب', 'rabiii', 'تختارو', 'يبزنسو', 'حسيبكم', 'ليغني', 'يهدوو', 'متواجده', 'تفهمو', 'rbo3', 'خطه', 'مختلفه', 'حظوظهم', 'امهات', 'ليله', 'فاسق', 'yestfazou', '4irhom', 'ونتعلموهم', 'moula', 'بالتصريح', 'khirlek', 'لمجابهه', 'sincère', 'للحل', 'مسيطرين', 'خلاهملنا', 'لجابتكم', 'yahfdhek', 'مفعولو', 'وحه', 'شغف', 'وقعدت', 'متباعدين', 'خييييير', 'تستطيعوا', '3bad', 'ضربه', 'ايجيه', 'thamer', 'تحشمش', 'بفففف', 'تتقدم', '3ases', 'fransa', 'مرجع', 'نعتبرها', 'ماقالوش', 'ينعمه', '3eyla', 'والتضحيات', 'but', 'twenssa', 'المسءوله', 'واله', 'للضمان', 'السنوات', 'مقلوبه', 'تحققلها', 'يصطاد', 'haw', 'ايامكم', 'w9ilit', 'ma5yebhaaaaa

In [173]:
vocab2=[]
for word in vocab:
    if len(word)>=3: #or word not in ['1','2','3','4','5','6','7','8','9','0']:
        vocab2.append(word)
print(len(vocab2))
    

27795


In [174]:
import re
vocab3=[]
for w in vocab2:
    vocab3.append(re.sub(r'(.)\1+', r'\1\1', w))
print(len(vocab3))

27795


In [175]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)

tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_seq = sequence.pad_sequences(X_train_seq, maxlen=max_len)

In [176]:
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)
#X_train_seq = tokenizer.texts_to_matrix(X_train, mode='freq')
print(X_train_seq.shape)
max_words=len(vocab3)
print(max_words)
print(max_len)
print(type(X_train))

(8834, 4430)
27795
4430
<class 'pandas.core.series.Series'>


In [177]:
print(X_train_seq)

[[    0     0     0 ...   152  4202    17]
 [    0     0     0 ...  7842  7843  7844]
 [    0     0     0 ...  7847  1037  7848]
 ...
 [    0     0     0 ...  2828  7652 28115]
 [    0     0     0 ...     1   119   251]
 [    0     0     0 ...   320   672  7687]]


In [178]:
train=X_train.values.tolist()
s=0
for i in range(len(train)):
    #print(train[i])
    s=s+len(train[i])
moy=s/len(train)
print(moy)

41.282884310618066


In [179]:
max_len=40

In [140]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

tokenizer = Tokenizer(num_words=max_words)

tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)


In [141]:
X_train_seq[1]

[49, 142, 1, 1509, 381]

In [142]:
X_train[1]

'karim gharbi masit i sitcom ma3jibnich'

In [143]:
X_train_seq = sequence.pad_sequences(X_train_seq, maxlen=max_len)

In [144]:
X_train_seq[2]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 7737, 1003, 7738])

In [145]:
X_train[2]

'mabldou'

In [146]:
print(X_train_seq[7])
print(X_train[7])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0 1004 1005]
 مدخل المقروض الاضراب يامعلم


In [147]:
Y_train

array([[0],
       [2],
       [1],
       ...,
       [0],
       [1],
       [3]], dtype=int64)

In [180]:
from sklearn.utils import class_weight as cw
# Calculate Class Weights
def get_weight(y):
    class_weight_current =  cw.compute_class_weight('balanced', np.unique(y), y)
    return class_weight_current

In [181]:
class_weight = get_weight(Y_train.flatten())

In [182]:
class_weight[3]

0.9881431767337807

In [183]:
def get_rnn_model():
    model = Sequential()
    
    model.add(Embedding(max_words, 50, input_length=max_len))
    model.add(LSTM(64))
    
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(256, activation='relu'))
    
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(4, activation='sigmoid'))
    
    model.summary()
    
    return model


In [184]:
model1 = get_rnn_model()

KeyError: 'metrics_6/accuracy/Identity'

In [185]:
def get_cnn_model():   
    model = Sequential()
    
    model.add(Embedding(max_words, 100, input_length=max_len))
    
    model.add(Conv1D(1024, 3, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    
    
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(2048, activation='relu'))
    
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(4, activation='sigmoid'))
    
    model.summary()
    return model

In [164]:
def plot_performance(history=None, figure_directory=None, ylim_pad=[0, 0]):
    xlabel = 'Epoch'
    legends = ['Training', 'Validation']

    plt.figure(figsize=(20, 5))

    y1 = history.history['acc']
    y2 = history.history['val_acc']

    min_y = min(min(y1), min(y2))-ylim_pad[0]
    max_y = max(max(y1), max(y2))+ylim_pad[0]


    plt.subplot(121)

    plt.plot(y1)
    plt.plot(y2)

    plt.title('Model Accuracy\n'+date_time(1), fontsize=17)
    plt.xlabel(xlabel, fontsize=15)
    plt.ylabel('Accuracy', fontsize=15)
    plt.ylim(min_y, max_y)
    plt.legend(legends, loc='upper left')
    plt.grid()

    y1 = history.history['loss']
    y2 = history.history['val_loss']

    min_y = min(min(y1), min(y2))-ylim_pad[1]
    max_y = max(max(y1), max(y2))+ylim_pad[1]


    plt.subplot(122)

    plt.plot(y1)
    plt.plot(y2)

    plt.title('Model Loss\n'+date_time(1), fontsize=17)
    plt.xlabel(xlabel, fontsize=15)
    plt.ylabel('Loss', fontsize=15)
    plt.ylim(min_y, max_y)
    plt.legend(legends, loc='upper left')
    plt.grid()
    if figure_directory:
        plt.savefig(figure_directory+"/history")

    plt.show()

In [165]:
from keras.layers import Input, Add, concatenate, Dense, Activation, BatchNormalization, Dropout, Flatten
from keras.layers import LeakyReLU, PReLU, Lambda, Multiply
from keras.layers import Embedding, LSTM, Bidirectional
from keras.models import Sequential
num_class = 4


KeyError: 'metrics_6/accuracy/Identity'

In [186]:
loss = 'categorical_crossentropy'
# loss = 'binary_crossentropy' only for binary classification
metrics = ['accuracy']

In [187]:
def date_time(x):
    if x==1:
        return 'Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x==2:    
        return 'Timestamp: {:%Y-%b-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x==3:  
        return 'Date now: %s' % datetime.datetime.now()
    if x==4:  
        return 'Date today: %s' % datetime.date.today() 

In [102]:
import time
import datetime
import keras.callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.optimizers import RMSprop, Adam
import keras

from keras.callbacks import TensorBoard

print("Starting...\n")

start_time = time.time()
print(date_time(1))

print("\n\nCompliling Model ...\n")
learning_rate = 0.001
optimizer = Adam(learning_rate)
# optimizer = Adam()

model1.compile(optimizer=optimizer, loss=loss, metrics=metrics)

verbose = 1
epochs = 100
batch_size = 128
validation_split = 0.2

print("Trainning Model ...\n")

callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        #min_delta=1e-2,
        min_delta=1,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=90,
        verbose=1,
    )
]

history1 = model1.fit(
    X_train_seq,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=verbose,
    callbacks=callbacks,
    validation_split=validation_split,
    class_weight =class_weight
    )

elapsed_time = time.time() - start_time
elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

print("\nElapsed Time: " + elapsed_time)
print("Completed Model Trainning", date_time(1))

Starting...

Timestamp: 2021-07-25 16:22:31


Compliling Model ...

Trainning Model ...



ValueError: Error when checking input: expected dense_38_input to have shape (26392,) but got array with shape (40,)

In [103]:
plot_performance(history=history1)

NameError: name 'history1' is not defined

In [188]:
num_class = 4
model2 = get_cnn_model(num_class=num_class)

TypeError: get_cnn_model() got an unexpected keyword argument 'num_class'

In [105]:
print("Starting...\n")

start_time = time.time()
print(date_time(1))

print("\n\nCompliling Model ...\n")
learning_rate = 0.001
optimizer = Adam(learning_rate)
# optimizer = Adam()

model2.compile(optimizer=optimizer, loss=loss, metrics=metrics)

verbose = 1
epochs = 100
batch_size = 128
validation_split = 0.2

print("Trainning Model ...\n")

history2 = model2.fit(
    X_train_seq,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=verbose,
    callbacks=callbacks,
    validation_split=validation_split,
    class_weight =class_weight
    )

elapsed_time = time.time() - start_time
elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

print("\nElapsed Time: " + elapsed_time)
print("Completed Model Trainning", date_time(1))

Starting...

Timestamp: 2021-07-25 16:40:06


Compliling Model ...

Trainning Model ...

Train on 7067 samples, validate on 1767 samples
Epoch 1/100
7067/7067 [==============================] - 18s 3ms/step - loss: 2.3462 - accuracy: 0.2829 - val_loss: 1.3714 - val_accuracy: 0.3237
Epoch 2/100
7067/7067 [==============================] - 16s 2ms/step - loss: 1.6871 - accuracy: 0.3962 - val_loss: 1.3523 - val_accuracy: 0.3599
Epoch 3/100
7067/7067 [==============================] - 16s 2ms/step - loss: 0.9477 - accuracy: 0.6356 - val_loss: 1.2925 - val_accuracy: 0.3384
Epoch 4/100
7067/7067 [==============================] - 16s 2ms/step - loss: 0.3574 - accuracy: 0.8678 - val_loss: 1.2013 - val_accuracy: 0.3860
Epoch 5/100
7067/7067 [==============================] - 16s 2ms/step - loss: 0.1660 - accuracy: 0.9441 - val_loss: 1.0389 - val_accuracy: 0.5529
Epoch 6/100
7067/7067 [==============================] - 16s 2ms/step - loss: 0.0778 - accuracy: 0.9755 - val_loss: 0.9233 - val_accu

Epoch 56/100
7067/7067 [==============================] - 16s 2ms/step - loss: 0.0192 - accuracy: 0.9950 - val_loss: 1.9322 - val_accuracy: 0.6955
Epoch 57/100
7067/7067 [==============================] - 16s 2ms/step - loss: 0.0212 - accuracy: 0.9935 - val_loss: 1.8767 - val_accuracy: 0.7080
Epoch 58/100
7067/7067 [==============================] - 16s 2ms/step - loss: 0.0265 - accuracy: 0.9938 - val_loss: 1.8866 - val_accuracy: 0.6995
Epoch 59/100
7067/7067 [==============================] - 16s 2ms/step - loss: 0.0158 - accuracy: 0.9948 - val_loss: 1.9551 - val_accuracy: 0.6933
Epoch 60/100
7067/7067 [==============================] - 16s 2ms/step - loss: 0.0160 - accuracy: 0.9956 - val_loss: 1.9173 - val_accuracy: 0.6882
Epoch 61/100
7067/7067 [==============================] - 15s 2ms/step - loss: 0.0178 - accuracy: 0.9942 - val_loss: 1.9233 - val_accuracy: 0.6904
Epoch 62/100
7067/7067 [==============================] - 16s 2ms/step - loss: 0.0219 - accuracy: 0.9941 - val_loss: 2

In [106]:
plot_performance(history=history2)

KeyError: 'acc'

<Figure size 1440x360 with 0 Axes>

In [107]:
test_X_seq = tokenizer.texts_to_sequences(X_test)
test_X_seq = sequence.pad_sequences(test_X_seq, maxlen=max_len)
accuracy1 = model1.evaluate(test_X_seq, Y_test)
accuracy2 = model2.evaluate(test_X_seq, Y_test)

ValueError: Error when checking input: expected dense_38_input to have shape (26392,) but got array with shape (40,)

In [108]:
print("Model Performance of RNN (Test Accuracy):")
print('Accuracy: {:0.2f}%\nLoss: {:0.3f}\n'.format(accuracy1[1]*100, accuracy1[0]))

print("\nModel Performance of RNN (Test Accuracy):")
print('v: {:0.2f}%\nLoss: {:0.3f}\n'.format(accuracy2[1]*100, accuracy2[0]))

Model Performance of RNN (Test Accuracy):


NameError: name 'accuracy1' is not defined

In [ ]:
ypreds1 = model1.predict_classes(test_X_seq, verbose=1)
ypreds2 = model2.predict_classes(test_X_seq, verbose=1)

In [ ]:
def plot_model_performace(result):
    sns.set_style("ticks")
    figsize=(22, 6)

    ticksize = 12
    titlesize = ticksize + 8
    labelsize = ticksize + 5

    xlabel = "Model"
    ylabel = "Score"

    title = "Model Performance"

    params = {'figure.figsize' : figsize,
              'axes.labelsize' : labelsize,
              'axes.titlesize' : titlesize,
              'xtick.labelsize': ticksize,
              'ytick.labelsize': ticksize}

    plt.rcParams.update(params)

    col1 = "model"
    col2 = "score"
    sns.barplot(x=col1, y=col2, data=result)
    plt.title(title.title())
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=90)
    plt.grid()
    plt.plot()
    plt.show()
    print(result)

In [ ]:
plot_confusion_matrix(Y_test, ypreds1, title="RNN")

In [ ]:
result = pd.DataFrame({'model': 'RNN', 'score': accuracy1[1]*100}, index=[-1])
row2 = pd.DataFrame({'model': 'CNN', 'score': accuracy2[1]*100}, index=[-1])
result = pd.concat([row2, result.ix[:]]).reset_index(drop=True)

In [ ]:
plot_model_performace(result)